In [ ]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
from collections import Counter
import numpy as np

news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1') # 뉴스 raw data
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1') # stock_list data
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1') # 대표단어 csv

# 테스트 용도로 데이터 1000개만 가지고
news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower() #소문자화
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
self_stop_words = {'bloomberg'}
news_raw['summary'].replace(self_stop_words, '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

news_raw['company_name'] = 0
news_raw['keyword'] = 0 # keybert로 뽑아낸 키워드 컬럼
news_raw['keyword2'] = 0

# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

# 맵핑한 데이터 저장하기 위한 csv파일 생성
project_result = pd.DataFrame(columns=['news_id', 'company_word','pk'])